In [2]:
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from langchain_openai import ChatOpenAI
import yfinance as yf
from dataclasses import dataclass
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Validate environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in environment variables")


@dataclass
class AnalysisConfig:
    """Configuration for analysis parameters"""

    period_years: int = 5
    interval: str = "1wk"  # 1d, 1wk, 1mo
    moving_averages: List[int] = None
    rsi_period: int = 14
    volume_ma_period: int = 20

    def __post_init__(self):
        if self.moving_averages is None:
            self.moving_averages = [50, 200]  # Default MAs


class Tasks:
    def research(self, agent):
        return Task(
            description="""Research the company with ticker {company} over the past 5 years. 
            1. Analyze major news events and their impact
            2. Track management changes and strategic shifts
            3. Evaluate competitive position changes
            4. Assess market sentiment trends
            5. Review analyst coverage and ratings history
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="""A comprehensive research report including:
                1. Timeline of major events and their market impact
                2. Changes in company strategy and execution
                3. Market sentiment analysis with specific examples
                4. Competitive position assessment
                5. Notable analyst opinions and rating changes""",
        )

    def technical_analysis(self, agent):
        return Task(
            description="""Perform detailed technical analysis for ticker {company} with 5-year historical data.
            1. Identify primary and secondary trends with SPECIFIC PRICE LEVELS
            2. Calculate EXACT support/resistance levels in numbers
            3. Analyze volume patterns with SPECIFIC percentage changes
            4. Calculate and interpret RSI, MACD values
            5. Compare current price to 50, 100, 200-day MAs
            6. Provide EXACT price targets
            YOU MUST PROVIDE SPECIFIC NUMBERS AND EXACT VALUES.""",
            agent=agent,
            expected_output="""A technical analysis report with EXACT numbers including:
                1. Current price: $XX.XX
                2. Primary trend: UP/DOWN with XX% change over past Y months
                3. Support levels: $XX.XX, $XX.XX
                4. Resistance levels: $XX.XX, $XX.XX
                5. RSI: XX.XX (OVERBOUGHT/NEUTRAL/OVERSOLD)
                6. Volume: XX% above/below 20-day average
                7. Price targets: 
                - Upside: $XX.XX (XX% potential)
                - Downside: $XX.XX (XX% risk)
                8. Clear BULLISH/BEARISH/NEUTRAL rating based on technicals""",
        )

    def financial_analysis(self, agent):
        return Task(
            description="""Conduct comprehensive financial analysis for company ticker {company}.
            1. Analyze 5-year trends in key metrics
            2. Evaluate capital structure and efficiency
            3. Assess profitability and growth metrics
            4. Review cash flow generation and usage
            5. Compare with industry benchmarks
            6. Examine insider transactions and ownership changes
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            expected_output="""A detailed financial analysis including:
                1. Growth trend analysis with specific metrics
                2. Profitability analysis with industry comparisons
                3. Balance sheet strength assessment
                4. Cash flow analysis and capital allocation
                5. Key risk factors identification
                6. Insider activity patterns and implications""",
        )

    def investment_recommendation(self, agent, context_tasks):
        return Task(
            description="""Synthesize all analyses to provide a detailed investment recommendation for {company}.
            1. Evaluate all technical and fundamental factors
            2. Assess risk-reward ratio
            3. Consider market conditions and timing
            4. Provide specific price targets
            5. Identify key catalysts and risks
            The company ticker is a required input parameter that will be provided.""",
            agent=agent,
            context_tasks=context_tasks,
            expected_output="""A specific investment recommendation including:
                1. Clear STRONG BUY / BUY / HOLD / SELL / STRONG SELL rating
                2. Specific price targets (entry, exit, stop-loss)
                3. Expected timeframe for the investment
                4. Key catalysts to monitor
                5. Risk factors and mitigation strategies
                6. Position sizing recommendations""",
        )


def save_analysis_to_markdown(result: str, ticker: str) -> Path:
    """Save the analysis result to a markdown file"""
    reports_dir = Path("reports")
    reports_dir.mkdir(exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = reports_dir / f"{ticker}_analysis_{timestamp}.md"

    content = f"""# Investment Analysis Report for {ticker}
Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

## Analysis Summary
{result}

---
*This report was generated automatically by the Stock Analysis System*
"""

    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)

    return filename


class FinancialMetrics:
    """Calculate financial metrics and ratios"""

    @staticmethod
    def calculate_ratios(financial_data: pd.DataFrame) -> Dict:
        try:
            current_ratio = (
                financial_data["Total Current Assets"]
                / financial_data["Total Current Liabilities"]
            )
            debt_to_equity = (
                financial_data["Total Liabilities"]
                / financial_data["Total Stockholder Equity"]
            )
            return {
                "current_ratio": current_ratio.iloc[-1],
                "debt_to_equity": debt_to_equity.iloc[-1],
                "quick_ratio": (
                    financial_data["Total Current Assets"] - financial_data["Inventory"]
                ).iloc[-1]
                / financial_data["Total Current Liabilities"].iloc[-1],
                "roe": (
                    financial_data["Net Income"]
                    / financial_data["Total Stockholder Equity"]
                ).iloc[-1],
            }
        except Exception as e:
            logger.error(f"Error calculating financial ratios: {e}")
            return {}


class TechnicalAnalysis:
    """Technical analysis calculations"""

    @staticmethod
    def calculate_rsi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        delta = data["Close"].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    @staticmethod
    def calculate_support_resistance(
        data: pd.DataFrame, window: int = 20
    ) -> Tuple[float, float]:
        rolling_min = data["Low"].rolling(window=window).min()
        rolling_max = data["High"].rolling(window=window).max()
        return rolling_min.iloc[-1], rolling_max.iloc[-1]


class ComprehensivePriceTool(BaseTool):
    name: str = "Comprehensive Price Analysis"
    description: str = "Get detailed historical price data with technical indicators"

    def __init__(self, config: AnalysisConfig):
        super().__init__()
        self._config = config

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(
                period=f"{self._config.period_years}y", interval=self._config.interval
            )

            if history.empty:
                return "No price data available for this ticker"

            # Calculate technical indicators
            for ma in self._config.moving_averages:
                history[f"MA_{ma}"] = history["Close"].rolling(window=ma).mean()

            history["RSI"] = TechnicalAnalysis.calculate_rsi(
                history, self._config.rsi_period
            )
            history["Volume_MA"] = (
                history["Volume"].rolling(window=self._config.volume_ma_period).mean()
            )

            support, resistance = TechnicalAnalysis.calculate_support_resistance(
                history
            )

            analysis_summary = {
                "price_data": history.to_csv(),
                "current_price": history["Close"].iloc[-1],
                "support_level": support,
                "resistance_level": resistance,
                "rsi": history["RSI"].iloc[-1],
                "volume_trend": (
                    "Up"
                    if history["Volume"].iloc[-1] > history["Volume_MA"].iloc[-1]
                    else "Down"
                ),
            }

            return str(analysis_summary)
        except Exception as e:
            logger.error(f"Error in price analysis: {e}")
            return f"Error analyzing price data: {str(e)}"


class FinancialMetricsTool(BaseTool):
    name: str = "Financial Metrics Analysis"
    description: str = "Analyze comprehensive financial metrics and growth trends"

    def _run(self, ticker: str) -> str:
        try:
            stock = yf.Ticker(ticker)

            # Get various financial statements
            balance_sheet = stock.balance_sheet
            income_stmt = stock.income_stmt
            cash_flow = stock.cashflow

            # Calculate key metrics and growth rates
            metrics = {
                "balance_sheet": balance_sheet.to_csv(),
                "income_statement": income_stmt.to_csv(),
                "cash_flow": cash_flow.to_csv(),
                "key_ratios": FinancialMetrics.calculate_ratios(balance_sheet),
                "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
                "profit_margin": (
                    income_stmt.loc["Net Income"] / income_stmt.loc["Total Revenue"]
                ).mean(),
            }

            return str(metrics)
        except Exception as e:
            logger.error(f"Error in financial analysis: {e}")
            return f"Error analyzing financial data: {str(e)}"


class AdvancedAgents:
    def __init__(self, config: AnalysisConfig):
        self.config = config

    def financial_analyst(self) -> Agent:
        return Agent(
            role="Senior Financial Analyst",
            goal="Provide comprehensive financial analysis focusing on long-term trends and company health",
            backstory="""A veteran financial analyst with 20 years of experience in equity research. 
                     Specialized in identifying long-term value opportunities through deep fundamental analysis.""",
            tools=[FinancialMetricsTool()],
            verbose=True,
        )

    def technical_analyst(self) -> Agent:
        return Agent(
            role="Technical Analysis Expert",
            goal="Analyze long-term price patterns and technical indicators to identify major trends",
            backstory="""A technical analysis expert with expertise in pattern recognition and trend analysis. 
                     Known for identifying major market turning points using multi-timeframe analysis.""",
            tools=[ComprehensivePriceTool(self.config)],
            verbose=True,
        )

    def hedge_fund_manager(self) -> Agent:
        return Agent(
            role="Hedge Fund Manager",
            goal="Synthesize all analyses to provide a clear investment recommendation",
            backstory="""A seasoned hedge fund manager with 25 years of experience in global markets.
              Knwon for making decisive investment calls on whether to 'Sell', 'Buy', or 'Hold' based on comprehensive analysis """,
            tools=[],  # Uses ananlysis from other agents
            verbose=True,
        )


def create_analysis_crew(
    ticker: str, config: AnalysisConfig = AnalysisConfig()
) -> Tuple[str, Path]:
    try:
        # Initialize agents and tasks
        agents = AdvancedAgents(config)
        tasks = Tasks()

        # Create analysts
        financial_analyst = agents.financial_analyst()
        technical_analyst = agents.technical_analyst()
        hedge_fund_manager = agents.hedge_fund_manager()

        # Define tasks
        financial_task = tasks.financial_analysis(financial_analyst)
        technical_task = tasks.technical_analysis(technical_analyst)

        # Final recommendation task uses results from previous analyses
        recommend_task = tasks.investment_recommendation(
            hedge_fund_manager,
            context_tasks=[financial_task, technical_task],
        )

        # Create and configure crew
        crew = Crew(
            agents=[financial_analyst, technical_analyst, hedge_fund_manager],
            tasks=[financial_task, technical_task, recommend_task],
            verbose=True,
            process=Process.sequential,
            manager_llm=ChatOpenAI(
                model_name="o3-mini",
                temperature=0.3,
                api_key=OPENAI_API_KEY,
            ),
        )

        # Execute analysis
        result = crew.kickoff(inputs={"company": ticker})

        # Save the result to markdown
        report_file = save_analysis_to_markdown(result, ticker)

        return result, report_file

    except Exception as e:
        logger.error(f"Error in analysis execution: {e}")
        raise


if __name__ == "__main__":
    try:
        # Configure analysis parameters
        config = AnalysisConfig(
            period_years=5,
            interval="1wk",
            moving_averages=[50, 100, 200],
            rsi_period=14,
            volume_ma_period=20,
        )

        # Run analysis
        result, report_file = create_analysis_crew("066570.KS", config)
        print(f"Analysis completed. Report saved to: {report_file}")
        print("\nAnalysis Result:")
        print(result)

    except Exception as e:
        logger.error(f"Analysis failed: {e}")

LLM value is None
LLM value is None
LLM value is None


21:12:03 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


# Agent: Senior Financial Analyst
## Task: Conduct comprehensive financial analysis for company ticker 066570.KS.
            1. Analyze 5-year trends in key metrics
            2. Evaluate capital structure and efficiency
            3. Assess profitability and growth metrics
            4. Review cash flow generation and usage
            5. Compare with industry benchmarks
            6. Examine insider transactions and ownership changes
            The company ticker is a required input parameter that will be provided.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:12:04 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
ERROR:__main__:Error calculating financial ratios: 'Total Current Assets'
C:\Users\Jonathan\AppData\Local\Temp\ipykernel_2644\1882708327.py:273: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  "revenue_growth": income_stmt.loc["Total Revenue"].pct_change().mean(),
21:12:05 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Senior Financial Analyst
## Thought: I need to analyze the company's financial metrics to gather comprehensive data on their performance and health.
## Using tool: Financial Metrics Analysis
## Tool Input: 
"{\"ticker\": \"066570.KS\"}"
## Tool Output: 
{'balance_sheet': ',2023-12-31,2022-12-31,2021-12-31,2020-12-31\r\nTreasury Shares Number,763176.0,763176.0,763176.0,763176.0\r\nPreferred Shares Number,17181299.0,17181299.0,17181299.0,17181300.0\r\nOrdinary Shares Number,162884638.0,162884638.0,162884638.0,162884638.0\r\nShare Issued,163647814.0,163647814.0,163652503.0,163647814.0\r\nNet Debt,4833248000000.0,4722827000000.0,3880037000000.0,4023446000000.0\r\nTotal Debt,14513834000000.0,12141422000000.0,10886975000000.0,10781815000000.0\r\nTangible Book Value,16711137070000.0,16451348070000.0,14701464000000.0,12212490070000.0\r\nInvested Capital,32899635070000.0,29951365070000.0,27076266000000.0,25271325070000.0\r\nWorking Capital,6180829000000.0,5155679000000.0,386786400000

...



# Agent: Technical Analysis Expert
## Using tool: Comprehensive Price Analysis
## Tool Input: 
"{\"ticker\": \"066570.KS\"}"
## Tool Output: 
{'price_data': 'Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MA_50,MA_100,MA_200,RSI,Volume_MA\r\n2020-02-03 00:00:00+09:00,62108.61699015602,65001.87554559807,62108.61699015602,64905.43359375,2528266,0.0,0.0,,,,,\r\n2020-02-10 00:00:00+09:00,64616.11328125,65966.30072294777,63748.13564015858,64616.11328125,2367365,0.0,0.0,,,,,\r\n2020-02-17 00:00:00+09:00,64519.66845924735,67509.36909039335,63458.80694496974,63748.1328125,3744172,0.0,0.0,,,,,\r\n2020-02-24 00:00:00+09:00,62494.39165976821,62687.27558464404,57865.17746274835,58250.9453125,3895394,0.0,0.0,,,,,\r\n2020-03-02 00:00:00+09:00,58154.50390625,59697.57532001451,57479.410162728025,58154.50390625,2936642,0.0,0.0,,,,,\r\n2020-03-09 00:00:00+09:00,58154.50446428572,59215.36607142857,48799.63392857143,50632.03125,4407479,0.0,0.0,,,,,\r\n2020-03-16 00:00:00+09:00,51210.68106464801,

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:12:17 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
21:12:17 - LiteLLM:INFO: utils.py:2784 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Technical Analysis Expert
## Final Answer: 
1. Current price: $78,400.00
2. Primary trend: DOWN with 14.46% change over the past 6 months
3. Support levels: $77,600.00, $76,000.00
4. Resistance levels: $113,700.00, $116,000.00
5. RSI: 32.08 (OVERSOLD)
6. Volume: Down 15% below 20-day average
7. Price targets: 
   - Upside: $100,000.00 (27.68% potential)
   - Downside: $76,000.00 (2.00% risk)
8. Rating: BEARISH based on technicals
```


# Agent: Hedge Fund Manager
## Task: Synthesize all analyses to provide a detailed investment recommendation for 066570.KS.
            1. Evaluate all technical and fundamental factors
            2. Assess risk-reward ratio
            3. Consider market conditions and timing
            4. Provide specific price targets
            5. Identify key catalysts and risks
            The company ticker is a required input parameter that will be provided.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:12:27 - LiteLLM:INFO: utils.py:941 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Hedge Fund Manager
## Final Answer: 
**Investment Recommendation for 066570.KS: STRONG BUY**

1. **Current Context**:  
   - **Current Price**: $78,400.00
   - **Primary Trend**: Currently down by 14.46% over the past 6 months, indicating bearish sentiment.
   - **Support Levels**: $77,600.00 and $76,000.00 are critical levels to watch.
   - **Resistance Levels**: $113,700.00 and $116,000.00, which provide substantial upside potential.

2. **Technical Analysis**:  
   - The stock is currently oversold with an RSI of 32.08, suggesting a potential rebound as it is priced below its intrinsic value.
   - Volume is down 15% below the 20-day average indicating lower interest but potentially setting up for a reversal as it appears more attractive to buyers at lower prices.

3. **Fundamental Analysis**:  
   - Although the technical indicators signal a bearish tone, the upside price target of $100,000.00 represents a significant potential gain of approximately 27.68%, against a down